In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.layers import Dense,Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout
from keras.utils import np_utils, print_summary
import tensorflow as tf
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import pickle
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [2]:
def keras_model(image_x, image_y):
    num_of_classes = 5
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(image_x,image_y,1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(num_of_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    filepath = "QuickDraw.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    return model, callbacks_list

In [3]:
def loadFromPickle(path):
    with open(path + "features", "rb") as f:
        features = np.array(pickle.load(f))
    with open(path + "labels", "rb") as f:
        labels = np.array(pickle.load(f))

    return features, labels

In [4]:
def augmentData(features, labels):
    features = np.append(features, features[:, :, ::-1], axis=0)
    labels = np.append(labels, -labels, axis=0)
    return features, labels

In [5]:
def prepress_labels(labels):
    labels = np_utils.to_categorical(labels)
    return labels

In [6]:
path = 'E:/Deep Learning/DeepProject/''
features, labels = loadFromPickle(path + 'DataSet/CNN_DataSet/')
features, labels = shuffle(features, labels)
labels=prepress_labels(labels)
train_x, test_x, train_y, test_y = train_test_split(features, labels, random_state=0, test_size=0.1)
train_x = train_x.reshape(train_x.shape[0], 28, 28, 1)
test_x = test_x.reshape(test_x.shape[0], 28, 28, 1)

In [7]:
model, callbacks_list = keras_model(28,28)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
__________

In [8]:
model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=10, batch_size=64, callbacks=[TensorBoard(log_dir="QuickDraw")])

Train on 45000 samples, validate on 5000 samples
Epoch 1/10
45000/45000 [==============================] - 21s 463us/step - loss: 0.4643 - acc: 0.8343 - val_loss: 0.2233 - val_acc: 0.9282
Epoch 2/10
45000/45000 [==============================] - 7s 164us/step - loss: 0.2288 - acc: 0.9284 - val_loss: 0.1861 - val_acc: 0.9410
Epoch 3/10
45000/45000 [==============================] - 7s 166us/step - loss: 0.1792 - acc: 0.9437 - val_loss: 0.1787 - val_acc: 0.9424
Epoch 4/10
45000/45000 [==============================] - 8s 168us/step - loss: 0.1523 - acc: 0.9524 - val_loss: 0.1632 - val_acc: 0.9486
Epoch 5/10
45000/45000 [==============================] - 8s 168us/step - loss: 0.1290 - acc: 0.9588 - val_loss: 0.1702 - val_acc: 0.9486
Epoch 6/10
45000/45000 [==============================] - 7s 167us/step - loss: 0.1163 - acc: 0.9619 - val_loss: 0.1656 - val_acc: 0.9490
Epoch 7/10
45000/45000 [==============================] - 7s 167us/step - loss: 0.0972 - acc: 0.9676 - val_loss: 0.1678 - 

In [9]:
model.save(path + 'QuickDraw/CNN__image_recognition/models/' + 'QuickDraw.h5')